In [1]:
import sys
import cmath
import math
import os

if sys.platform=="cygwin":
    from cyglibra_core import *
elif sys.platform=="linux" or sys.platform=="linux2":
    from liblibra_core import *

from libra_py import units
from libra_py import data_conv
from libra_py import QE_methods

import libra_py.workflows.librax.defaults as defaults
import libra_py.workflows.librax.main as main

import libra_py.workflows.librax.x_to_libra_qe as x_to_libra_qe
import libra_py.workflows.librax.create_input_qe as create_input_qe
import libra_py.workflows.librax.extract_qe as extract_qe


import matplotlib.pyplot as plt   # plots
#import numpy as np
#from matplotlib.mlab import griddata

%matplotlib inline 

plt.rc('axes', titlesize=24)      # fontsize of the axes title
plt.rc('axes', labelsize=20)      # fontsize of the x and y labels
plt.rc('legend', fontsize=20)     # legend fontsize
plt.rc('xtick', labelsize=16)    # fontsize of the tick labels
plt.rc('ytick', labelsize=16)    # fontsize of the tick labels

plt.rc('figure.subplot', left=0.2)
plt.rc('figure.subplot', right=0.95)
plt.rc('figure.subplot', bottom=0.13)
plt.rc('figure.subplot', top=0.88)

colors = {}

colors.update({"11": "#8b1a0e"})  # red       
colors.update({"12": "#FF4500"})  # orangered 
colors.update({"13": "#B22222"})  # firebrick 
colors.update({"14": "#DC143C"})  # crimson   

colors.update({"21": "#5e9c36"})  # green
colors.update({"22": "#006400"})  # darkgreen  
colors.update({"23": "#228B22"})  # forestgreen
colors.update({"24": "#808000"})  # olive      

colors.update({"31": "#8A2BE2"})  # blueviolet
colors.update({"32": "#00008B"})  # darkblue  

colors.update({"41": "#2F4F4F"})  # darkslategray

clrs_index = ["11", "21", "31", "41", "12", "22", "32", "13","23", "14", "24"]

/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for std::vector<std::complex<double>, std::allocator<std::complex<double> > > already registered; second conversion method ignored.
  if __name__ == '__main__':
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > > already registered; second conversion method ignored.
  if __name__ == '__main__':
/home/alexey/miniconda2/envs/libra/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: to-Python converter for boost::python::detail::container_element<std::vector<std::vector<int, std::allocator<int> >, std::allocator<std::vector<int, std::allocator<int> > > >, unsigned long, boost::python::detail::final_vector_derived_policies<std::vector<std::vector<int, std::allocator<in

In [2]:
params = {}

params["nspin"] = 1
params["electronic_smearing"] = 0.01 # Electronic smearing used in Fermi population calculation
params["scf_iter"] = 1


params["EXE"] = "/mnt/c/cygwin/home/Alexey-user/Soft/espresso/bin/pw.x" 
params["EXE_EXPORT"] = "/mnt/c/cygwin/home/Alexey-user/Soft/espresso/bin/pw_export.x"
params["BATCH_SYSTEM"] = None
params["NP"] = 1


params["excitations"] = [ excitation(0,1,0,1), excitation(0,1,1,1) ] 

In [3]:
def deltaSCF(ex_st, params):
    
    # Make a temporary working input file
    os.system("cp x%i.scf.in x%i.scf_wrk.in" % (ex_st, ex_st))
        

    print "Starting QE calculations"
    x_to_libra_qe.exe_espresso(ex_st, params)
    status = x_to_libra_qe.check_convergence("x%i.scf.out" % (ex_st))

    print "Status = ", status
    
    info = QE_methods.read_qe_schema("x%i.save/data-file-schema.xml" % (ex_st))
    print "Energy = ", info["etot"]
    
    excitation = params["excitations"][ex_st]
    nspin = params["nspin"]
    nel = info["nelec"]
    norbs = info["nbnd"]/nspin  # the number of orbitals per spin channel
    kT = params["electronic_smearing"]

        
    occ, occ_alp, occ_bet = create_input_qe.excitation_to_qe_occ(norbs, nel, excitation)
    
    print "Original occupation numbers :"
    if nspin==1:
        print occ
    elif nspin==2:
        print occ_alp, occ_bet

        
    # In the case we didn't converge, lets do the iterations with the occupation numbers
    cnt = 0
    restart_flag = 1   #  1 - change scf_iter, 2 - change scf_iter + restart from the previous pot and wfc    

    while status == False and cnt<10:                 
        print  "In the while loop count = %i" % (cnt) 

        #================== Write the file =========================
        create_input_qe.write_qe_input_first("x%i.scf_wrk.in" % (ex_st), "x%i.scf_wrk.in" % (ex_st),
                                         occ, occ_alp, occ_bet, nspin, params["scf_iter"], restart_flag)
                
        #=================== Run the QE calculations and get the info ==========
        print "starting QE"
        x_to_libra_qe.exe_espresso(ex_st, params)
        status = x_to_libra_qe.check_convergence("x%i.scf.out" % (ex_st))

        print "Status = ", status
        info = QE_methods.read_qe_schema("x%i.save/data-file-schema.xml" % (ex_st), 0)
        print "Energy = ", info["etot"]

        if cnt > 0:  
            restart_flag = 2
            
        nbnd = info["nbnd"]

        
        #============== Recompute the populations =======================
        if nspin == 1:
            info1, all_e = QE_methods.read_qe_index("x%i.export/index.xml" % (ex_st), range(1, nbnd + 1), 0)

            bnds = []
            for i in xrange(nbnd):                    
                bnds.append(all_e[0].get(i,i).real)
            occ = extract_qe.excited_populations(bnds, info["nelec"], nspin, kT, 1)
            print "Energies = ", bnds
            print "Updated occ = ", occ


        elif nspin == 2:
            info1, all_e = QE_methods.read_qe_index("x%i.export/index.xml" % (ex_st), range(1, nbnd/2 + 1), 0)

            bnds_alp, bnds_bet = [], []
            for i in xrange(nbnd/2):                    
                bnds_alp.append(all_e[0].get(i,i).real)
                bnds_bet.append(all_e[1].get(i,i).real)
            occ_alp = extract_qe.excited_populations(bnds_alp, info["nelec"], nspin, kT, 1)
            occ_bet = extract_qe.excited_populations(bnds_bet, info["nelec"], nspin, kT, 1)
            print "Energies = ", bnds_alp, bnds_bet
            print "Updated occ = ", occ_alp, occ_bet

        cnt = cnt + 1

    print status


In [4]:
deltaSCF(1, params)

Starting QE calculations
Status =  False
Energy =  -14.542779638
Original occupation numbers :
[2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
In the while loop count = 0
starting QE
Status =  False
Energy =  -14.5026605952
path= Root
Energies =  [-0.875329403997472, -0.66856295221974, -0.570182401333897, -0.545615220392482, -0.4502432509577524, -0.3887823034829896, -0.1640168245215736, -0.06084055234649025, -0.01356460469086948, 0.007248730177921535, 0.08254133881128825, 0.03004284734418239, 0.01836580971895045, 0.03387382450249343, 0.01571750081659748, 0.07461693876231085]
Updated occ =  [2.0, 1.9999999999985985, 1.9999999737435346, 1.9999996936800288, 1.995761560209713, 1.0042651180661981, 0.9999068467970186, 6.607898007547672e-05, 5.846678781682276e-07, 7.294536420438817e-08, 3.127545743115997e-08, 2.3998775200792618e-08, 7.465549522591736e-09, 5.089610610978059e-09, 8.654332731504274e-11, 3.9181518840472485e-11]
In the while loop count = 1
starting